# Маркетинг

## 1. Общие данные

**Описание:**
Интернет-магазин собирает историю покупателей, проводит рассылки предложений и планирует будущие продащи. Для оптимизации процессов надо выделить пользователей, которые готовы совершить покупку в ближайшее время.

**Цель:**
Предсказать вероятность покупки в течении 90 дней

**Задачи:**
- Изучить данные;
- Разработать полезные признаки;
- Создать модель для классификации пользователей;
- Улучшить модель и максимизировать метрику roc-auc;
- Выполнить тестирование.

**Исходные данные:**
- Файл с историей покупок;

- Файл с историей рекламных рассылок;

- Файл с информацией: соверишит ли клиент покупку в течении следующих 90 дней;

- Файл с агрегицией общей базы рассылок по дням и типам событий;

- Файл с агрегацией по дням с учетом событий и каналов рассылки.

**План проекта:**

1) Загрузка исходных данных.

2) Изучение исходных данных. Обработка  некоректных типов данных, пропущенных значений, дубликатов, аномалий. 

3) Разработка полезных признаков. 

4) Подготовка данных для обучения моделей.

5) Обучение моделей для классификации пользователей ("Обучение с учителем") с основной метрикой roc-auc.

6) Улучшение модели и максимизация метрики roc-auc.

7) Тестирование лучшей модели.

## 2. Настройка и подготовка

Импортируем необходимые библиотеки

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import phik
import time

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    RobustScaler
)
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# Настройки pandas
pd.set_option('display.float_format', '{:,.2f}'.format)

Зададим константы

In [ ]:
RANDOM_STATE = 42
TEST_SIZE = 0.25

Зададим функции

Функции для загрузки данных

In [ ]:
# Функция для загрузки файлов CSV

def load_csv(filepath, parse_dates=None):
    """
    Вход: filepath - путь к файлу
          parse_dates - столбцы с датами
    Выход: датасет
    
    """
    if os.path.exists(filepath):
        try:
            df = pd.read_csv(filepath, parse_dates=parse_dates)
            print(f"{filepath} успешно загружен")
            return df
        except Exception as e:
            print(f"Ошибка при загрузке {filepath}: {e}")
            return None
    else:
        print(f"Файл {filepath} не найден")
        return None
    
# Функция для вывода первых строк 
# и размера датасета

def head_shape(df, num_str=5):
    """
    Вход: df - датасет
          num_str- количество выводимых строк
    Выход: первые n строк и размер датасета

    """
    display(df.head(num_str))
    print(df.shape)
    print('')
    return


Функции для анализа данных

In [ ]:
# Функция для отображения пропусков
def miss_dupl(df):
    """
    Вход: df-датасет
    Выход: количество пропусков и дубликатов
    
    """
    print(f'Пропущенных значений:\n{df.isna().sum()}\n')
    
    print(f'Явных дубликатов:\n{df.duplicated().sum()}')
    
# Функция для просмотра уникальных значений
def uniq(df):
    """
    Ввод: df-датасет
    Вывод: уникальные значения столбца
    """
    for column in df.columns:
        print(f'{column}, {df[column].unique()}\n')
        
# Функция вывода общих данных 
def desc(df):
    """
    Ввод: df-датасет
    Вывод: общая информация о датасете
    """
    display(df.describe())
    if df.select_dtypes(include=[object]).shape[1] > 0:
        display(df.describe(include=[object]))

Функции для построения графиков

In [ ]:
# Функция для построения гистограмм
def hist(df, col, bins, target=None, col_names=None):
    """
    Ввод: 
        df - датасет; 
        col - столбец; 
        bins - количество корзин; 
        target - целевая переменная;
        col_names - словарь с названиями столбцов 
    Вывод: гистограмма распределения признака с разбивкой по целевому признаку
    """
    sns.set()
    plt.figure(figsize=(10, 8))
    
    # Получаем читаемое название для оси X
    x_label = col_names.get(col, col) if col_names else col
    
    # Строим гистограмму
    if target is not None:
        # Получаем читаемое название для легенды
        hue_label = col_names.get(target, target) if col_names else target
        sns.histplot(df, bins=bins, hue=target, x=col)
        plt.legend(title=hue_label)
    else:
        sns.histplot(df, bins=bins, x=col)
    
    plt.title(f'Распределение признака {x_label}', fontsize=16)
    plt.xlabel(x_label, fontsize=14)
    plt.ylabel('Частота', fontsize=14)
    plt.show()

# Функция для построения гистограмм совместно с графиком ящик с усами
def hist_box(df, col, bins, target=None, col_names=None):
    """
    Ввод: 
        df - датасет; 
        col - столбец; 
        bins - количество корзин; 
        target - целевая переменная;
        col_names - словарь с названиями столбцов 
    Вывод: гистограмма распределения признака и график ящик с усами
    """
    sns.set()
    f, axes = plt.subplots(2, 1, figsize=(10, 8))  
    
    # Получаем читаемое название для оси X
    x_label = col_names.get(col, col) if col_names else col
    
    # Верхний график - гистограмма
    axes[0].set_title(f'Распределение признака {x_label}', fontsize=16)
    axes[0].set_ylabel('Частота', fontsize=14)
    axes[0].set_xlabel(x_label, fontsize=14)  # ДОБАВЛЕНО: подпись оси X для верхнего графика
    
    if target is not None:
        # Получаем читаемое название для легенды
        hue_label = col_names.get(target, target) if col_names else target
        sns.histplot(df, bins=bins, ax=axes[0], hue=target, x=col)
        axes[0].legend(title=hue_label)
    else:
        sns.histplot(df, bins=bins, ax=axes[0], x=col)
    
    # Нижний график - ящик с усами
    axes[1].set_title(f'Ящик с усами для признака {x_label}', fontsize=16)
    sns.boxplot(data=df, ax=axes[1], x=col)
    axes[1].set_xlabel(x_label, fontsize=14)
    axes[1].set_ylabel('Значения', fontsize=14)
    
    plt.tight_layout() 
    plt.show()

# Функция для построения столбчатой диаграммы
def countplot(df, col, target=None, col_names=None):
    """
    Ввод: 
        df - датасет; 
        col - столбец; 
        target - целевая переменная;
        col_names - словарь с названиями столбцов 
    Вывод: столбчатая диаграмма распределения с разбивкой по целевому признаку
    """
    plt.figure(figsize=(10, 8))
    
    # Получаем читаемые названия
    x_label = col_names.get(col, col) if col_names else col
    
    plot = sns.countplot(data=df, x=col, hue=target)
    plot.set_title(f'Распределение по {x_label}', fontsize=16)
    plot.set_ylabel('Количество', fontsize=14)
    plot.set_xlabel(x_label, fontsize=14)
    
    # Добавляем поворот подписей если они длинные
    if df[col].nunique() > 5:
        plt.xticks(rotation=45, ha='right')
    
    # Добавляем легенду если есть target
    if target is not None:
        hue_label = col_names.get(target, target) if col_names else target
        plt.legend(title=hue_label)
    
    plt.tight_layout()
    plt.show()

# Функция для построения диаграммы рассеяния
def scatter(df, col, target, col_names=None):
    """
    Ввод: 
        df - датасет; 
        col - столбец; 
        target - целевая переменная;
        col_names - словарь с человекочитаемыми названиями столбцов (опционально)
    Вывод: диаграмма рассеяния col от target
    """
    plt.figure(figsize=(10, 6))
    
    # Получаем читаемые названия для осей
    x_label = col_names.get(col, col) if col_names else col
    y_label = col_names.get(target, target) if col_names else target
    
    sns.scatterplot(
        data=df, 
        x=col, 
        y=target,
        alpha=0.7,  
        s=50        
    )
    plt.title(f'Диаграмма рассеяния: {x_label} vs {y_label}', fontsize=16)
    plt.xlabel(x_label, fontsize=14)
    plt.ylabel(y_label, fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.show()

Функция для построения корреляционного анализа

In [ ]:
def phik_mat(df, interval_cols=None):
    """
    Ввод: df - датасет; 
    interval_cols - список с интервальными признаками;
    
    Вывод: матрица корреляции Phik
    """
    phik_matrix = df.phik_matrix(verbose=False, interval_cols=interval_cols)
    
    mask = np.triu(np.ones_like(phik_matrix, dtype=bool))
    
    plt.figure(figsize=(20, 15))
    sns.heatmap(
        phik_matrix, 
        annot=True, 
        fmt='.2f',
        cmap="coolwarm",
        mask=mask)
    plt.title('Корреляционная матрица (Phik)', fontsize=18)
    plt.show()

Функция для вывода результатов обучения модели на кросс-валидации

In [ ]:
def print_best_metrics(cv_results):
    """
    Выводит метрики лучшей модели по ROC-AUC из результатов кросс-валидации.
    
    Parameters:
    -----------
    cv_results : pandas DataFrame
        Результаты кросс-валидации из GridSearchCV/RandomizedSearchCV
    """
    best_roc_idx = cv_results['mean_test_roc_auc'].idxmax()
    
    best_metrics = {
        'roc_auc': {
            'mean': cv_results.loc[best_roc_idx, 'mean_test_roc_auc'],
            'std': cv_results.loc[best_roc_idx, 'std_test_roc_auc']
        },
        'f1': {
            'mean': cv_results.loc[best_roc_idx, 'mean_test_f1'],
            'std': cv_results.loc[best_roc_idx, 'std_test_f1']
        },
        'precision': {
            'mean': cv_results.loc[best_roc_idx, 'mean_test_precision'],
            'std': cv_results.loc[best_roc_idx, 'std_test_precision']
        },
        'recall': {
            'mean': cv_results.loc[best_roc_idx, 'mean_test_recall'],
            'std': cv_results.loc[best_roc_idx, 'std_test_recall']
        }
    }
    
    print("=" * 50)
    print("МЕТРИКИ ЛУЧШЕЙ МОДЕЛИ (по ROC-AUC)")
    print("=" * 50)
    for metric_name, values in best_metrics.items():
        print(f"{metric_name.upper():12} | Среднее: {values['mean']:.4f}")

## 3. Загрузка исходных данных

Создадим переменные для загрузки данных

In [ ]:
# Переменные pth_ - пути до файла, 
# переменные column_date_ - столбцы с датами
pth_1 = 'initial_data/apparel-messages.csv'
column_date_1 = ['date', 'created_at']

pth_2 = 'initial_data/apparel-purchases.csv'
column_date_2 = ['date']

pth_3 = 'initial_data/apparel-target_binary.csv'

pth_4 = 'initial_data/full_campaign_daily_event.csv'
column_date_4 = ['date']

pth_5 = 'initial_data/full_campaign_daily_event_channel.csv'
column_date_5 = ['date']

In [ ]:
# Загрузим исходные файлы в датафреймы
initial_messages = load_csv(pth_1, column_date_1 )
initial_purchases = load_csv(pth_2, column_date_2)
initial_target = load_csv(pth_3, None)
daily_event = load_csv(pth_4, column_date_4)
daily_chanel_event = load_csv(pth_5, column_date_5)

Проверим датасеты

In [ ]:
# Создадим словарь с именами и датасетами
data = {
    'initial_messages': initial_messages,
    'initial_purchases': initial_purchases,
    'initial_target': initial_target,
    'daily_event': daily_event,
    'daily_chanel_event': daily_chanel_event
}

In [ ]:
# Выведем первые пять строк и размер датасета
# Используем цикл для перебора
for name, df in data.items():
    print(f"Датасет: {name}")
    head_shape(df)

Все данные отображаются корректно

Проверим общую информацию о датасетах

In [ ]:
for name, df in data.items():
    print('')
    print(f"Датасет: {name}")
    df.info()

### 3.1 Промежуточный вывод

Данные заружены, установлена настройка при чтении файла, для обработки столбцов с датами. Наименования столбцов имеют "змеиный" регистр. Все данные имеют корректный тип данных.

## 4. Изучение исходных данных


### 4.1 Общая статистика

Выведем общую статистику по столбцам

#### 4.1.1 Данные о рассылках

In [ ]:
desc(initial_messages)

По общей статистике рассылок видно, что столбец `date` и `created_at` один и тот же столбец, только один с точным временем, а другой без. А также видно, что у нас два уникальных канала коммуникации, самый популярный из них пуш-уведомления на телефоне. Есть 11 уникальных действий, самым популярным является `send`. Датасет содержит данные за период с 19.12.2022 по 15.02.2024 год.

#### 4.1.2 Данные о покупках

In [ ]:
desc(initial_purchases)

По статистике данных о покупке видно, что в среднем покупатель покупает 1 единицу товара, максимальное значение 30 единиц товара в одном чеке. У нас есть данный за период с 16.05.2022 по 16.02.2024. Самая маленька сумма покупки равна 1, а сумма самого большого чека 85499.

#### 4.1.3 Данные с информацией, совершил ли покупку клиент в течении 90 дней

In [ ]:
desc(initial_target)

Видно, что только 2 процента совершили покупку в срок 90 дней.

#### 4.1.4 Агрегированные данные рассылок по дням и типам события, также агрегированные данные рассолок по дням с учетом событий и каналов рассылки

In [ ]:
# Зададим инструкцию для вывода всех столбцов
with pd.option_context('display.max_columns', None):
    desc(daily_event)

In [ ]:
# Зададим инструкцию для вывода всех столбцов
with pd.option_context('display.max_columns', None):
    desc(daily_chanel_event)

В данных файлах собраны агрегированные данные, `count_event*` содержит общее количество каждого события, `nunique_event*` содержит количество уникальных пользователей в каждом событии, `count_event*_channel*` содержит общее количество каждого события по каналам, `nunique_event*_channel*` содержит количество уникальных пользователей по событиям и каналам. 

### 4.2 Неявные дубликаты 

In [ ]:
initial_messages['event'].value_counts()

In [ ]:
initial_messages['channel'].value_counts()

Неявных дубликатов не обнаружено

### 4.3 Аномальные значения

In [ ]:
col_name = {'quantity': 'количество единиц товара',
            'price': 'цена товара'}

countplot(initial_purchases, 'quantity', None, col_name)

В основном в чеке только 1 единица товара. Проверим строки в которых в чеке более 1 единицы товара

In [ ]:
initial_purchases.loc[
    initial_purchases['quantity'] > 1,
      'quantity'
      ].value_counts()

Видно, что в чеках встречаются разное количество товаров, аномальные значения отсутствуют.

In [ ]:
hist_box(initial_purchases, 'price', 30, None, col_name)

Видно, что основное количество товаров стоит до 10 тысяч. Проверим сроки с большим значением цены товара

In [ ]:
# Отсортируем строки с ценой выше 30'000
initial_purchases.loc[initial_purchases['price'] > 30000]

Видно, что в данных строках количество товаров равно 1. Также видно чеки с одинаковыми категориями, но с разной ценой, это может быть связано со скидкой или повышении цен в зависимости от даты. Посмотрим количество строк с ценой выше 10000.

In [ ]:
# Отсортируем строки с ценой выше 10'000
more_ten = initial_purchases.loc[
    initial_purchases['price'] > 10000,
      'price'
      ].count()
# Найдем долю строк с ценой выше 10'000 к общему количеству
(more_ten / initial_purchases.shape[0]) * 100

Доля строк с ценой выше 10000 составляет 0,16 % от общего количества строк. Удалим данные строки, для исключения смещения распределения в сторону большей цены.

In [ ]:
# Создадим копию исходного датасета
purchases = initial_purchases.copy()

# Удалим строки с ценой более 10'000
purchases = purchases[purchases['price'] < 10000]

Проверим распределение по цене в обновленном датасете

In [ ]:
hist_box(purchases, 'price', 30, None, col_name)

Мы установили порог по цене одного товара в 10'000. Большенство строк имеют значение цены до 4'000.

### 4.4 Пропущенные значения и явные дубликаты

Создадим копии датасетов, которые будем обрабатывать

In [ ]:
messages = initial_messages.copy()
target = initial_target.copy()
event = daily_event.copy()
chanel_event = daily_chanel_event.copy()

In [ ]:
# Создадим словарь с датасетами для обработки
prep_df = {
    'messages': messages,
    'purchases': purchases,
    'target': target,
    'event': event,
    'chanel_event': chanel_event
}

Проверим пропущенные значения и полные дубликаты

In [ ]:
for name, df in prep_df.items():
    print('')
    print(f"Датасет: {name}")
    miss_dupl(df)

Пропущенные значения отсутствуют, но в датасетах `messages` и `purchases` большое количество явных дубликатов. Это большая ошибка, т.к. по данным датасетам можно считать выручку, задвоение строк может внести кассовый разрыв, между зафиксированной выручкой и фактической. Удалим полные дубликаты

In [ ]:
messages = messages.drop_duplicates()

In [ ]:
purchases = purchases.drop_duplicates()

### 4.5 Потери при предобработке данных 

Посмотрим, сколько мы потеряли данных при предобработке данных

Датасет `messages`

In [ ]:
round(
    100 - (messages.shape[0] / initial_messages.shape[0]
           ) * 100, 2)

Датасет `purchases`

In [ ]:
round(
    100 - (purchases.shape[0] / initial_purchases.shape[0]
           ) * 100, 2)

Датасет `target`

In [ ]:
round(
    100 - (target.shape[0] / initial_target.shape[0]
           ) * 100, 2)

Датасет `event`

In [ ]:
round(
    100 - (event.shape[0] / daily_event.shape[0]
           ) * 100, 2)

Датасет `chanel_event`

In [ ]:
round(
    100 - (chanel_event.shape[0] / daily_chanel_event.shape[0]
           ) * 100, 2)

В датасете `purchases`  36.22 % удаленных значений. Основное количество удаленных строк являются полные дубликаты. Возможно один клиент покупал один и тот же товар несколько раз в один день, но мы исключаем данный вариант, так как количество таких дубликатов очень велико.

### 4.6 Промежуточный вывод

Мы выполнили первичную обработку данных. 
В данных отсутствуют пропущенные значения. В датасете `purchases` удалены аномальные значения цены единицы товара (установлена верхняя граница 10000), а также удалены полные дубликаты - 72900 строк из 202208.

## 5. Разработка полезных признаков

**Датасет Purchases**

Найдем последнюю дату покупки

In [ ]:
last_date = purchases['date'].max()

Разобьем датe покупки на день недели, месяц, час, год

In [ ]:
purchases['purchase_weekday'] = purchases['date'].dt.weekday
purchases['purchase_hour'] = purchases['date'].dt.hour
purchases['purchase_month'] = purchases['date'].dt.month
purchases['purchase_year'] = purchases['date'].dt.month

Создадим новые признаки на основе даты

In [ ]:
# Добавим новые признаки:
# 1. В какой день недели чаще всего были покупки
# 2. В какой час чаще всего были покупки
# 3. Склонность покупок в выходные дни

time_features = purchases.groupby('client_id').agg(
    favorite_weekday=(
        'purchase_weekday', 
        lambda x: x.mode()[0] if not x.mode().empty else -1),
    favorite_hour=(
        'purchase_hour', 
        lambda x: x.mode()[0] if not x.mode().empty else -1),
    weekend_purchases_ratio=(
        'purchase_weekday', 
        lambda x: ((x >= 5).sum() / len(x)) if len(x) > 0 else 0)
).reset_index()

Добавим общую стоимость покупки

In [ ]:
purchases['total_price'] = \
purchases['quantity'] * purchases['price']

Создадим дополнительные признаки

In [ ]:
# Объединим данные по client_id и с помощью agg добавим признаки
# 1. Количество дней с последней покупки
# 2. Количество дней с покупками
# 3. Общая сумма покупок
# 4. Средняя сумма покупок
# 5. Максимальная стоимость покупки
# 6. Общее количество товаров
# 7. Среднее количество товаров

purchases_new_features = purchases.groupby('client_id').agg(
    days_since_last_purchase=(
        'date', lambda x: (last_date - x.max()).days),
    purchase_frequency=('date', 'nunique'),
    total_spent=('total_price', 'sum'),
    avg_purchase_value=('total_price', 'mean'),
    max_purchase=('total_price', 'max'),
    total_items=('quantity', 'sum'),
    avg_items_per_order=('quantity', 'mean')
).reset_index()

Преобразуем столбец `category_ids` в более удобные и понятные признаки

In [ ]:
# Преобразуем строки категорий в список
purchases['cat_list'] = purchases[
    'category_ids'
    ].str.strip("[]").str.replace("'", "").str.split(", ")

Сохраним последнюю и предпоследнюю нумерацию, т.к. они самые информативные для определенного товара. А также сохраним количество категорий

In [ ]:
purchases['last_category'] = purchases[
    'cat_list'
    ].apply(lambda x: x[-1] if len(x) >= 1 else None)

purchases['second_last_category'] = purchases[
    'cat_list'
    ].apply(lambda x: x[-2] if len(x) >= 2 else None)

# Количество категорий в списке
purchases['category_count'] = purchases['cat_list'].apply(len)

Создадим новые признаки на основе категорий товаров

In [ ]:
# Объединим данные по client_id и с помощью agg добавим признаки
# 1. Количество уникальных последних категорий
# 2. Количество уникальных предпоследних категорий
# 3. Самая частая последняя категория товара
# 4. Самая частая предпоследняя категория товара
# 5. Среднее количество категорий в покупке
# 6. Максимальное количество категорий в покупке
# 7. Минимальное количество категорий в покупке
# 8. Общее количество категорий во всех покупках


category_features = purchases.groupby('client_id').agg(
    unique_last_categories=('last_category', 'nunique'),
    unique_second_last_categories=('second_last_category', 'nunique'),
    most_frequent_last_category=('last_category', lambda x: x.mode()[0] if not x.mode().empty else None),
    most_frequent_second_last_category=('second_last_category', lambda x: x.mode()[0] if not x.mode().empty else None),
    avg_categories_per_purchase=('category_count', 'mean'),
    max_categories_in_purchase=('category_count', 'max'),
    min_categories_in_purchase=('category_count', 'min'),   
    total_categories=('category_count', 'sum')
).reset_index()

**Датасет messages**

Создадим новые признаки на основе сообщений

In [ ]:
# Сгруппируем данные по `client_id` и создадим признаки
# 1. Общее количество сообщений
# 2. Количество открытых сообщений
# 3. Количество сообщений с кликами
# 4. Количество покупок из сообщений
# 5. Количесвто уникальных компаний
# 6. Количество дней с последнего сообщения
msg_features = messages.groupby('client_id').agg(
    total_messages=('event', 'size'),
    opened_count=('event', lambda x: (x == 'opened').sum()),
    clicked_count=('event', lambda x: (x == 'clicked').sum()),
    purchased_from_msg=('event', lambda x: (x == 'purchase').sum()),
    unique_campaigns=('bulk_campaign_id', 'nunique'),
    days_since_last_msg=('date', 
                         lambda x: (
                             last_date - x.max()
                             ).days if len(x) > 0 else 365)
).reset_index()

Создадим признаки на основе новых признаков

In [ ]:
# 1. Доля сообщений, которые открыли
# 2. Доля кликов
# 3. Доля покупок
# 4. Конверсия открытых сообщений в клики

msg_features['open_rate'] = \
    msg_features['opened_count'] / msg_features[
        'total_messages'].replace(0, 1)

msg_features['click_rate'] = \
    msg_features['clicked_count'] / msg_features[
        'total_messages'].replace(0, 1)

msg_features['purchase_rate'] = \
    msg_features['purchased_from_msg'] / msg_features[
        'total_messages'].replace(0, 1)

msg_features['click_to_open'] = \
    msg_features['clicked_count'] / msg_features[
        'opened_count'].replace(0, 1)

Объединим все новые признаки и целевой признак в один датасет

In [ ]:
# Создадим список с новыми признаками
features_list = [
    purchases_new_features, 
    category_features, 
    msg_features, 
    time_features
    ]

all_features = purchases_new_features

# Объединим все новые признаки в один датасет
for feature_df in features_list[1:]:
    all_features = pd.merge(all_features, feature_df, on='client_id', how='left')

Выведем все признаки

In [ ]:
all_features.columns

Объединим все новые признаки с целевой переменной

In [ ]:
final_data = pd.merge(all_features, target, on='client_id', how='inner')

Выведем размер нового датасета

In [ ]:
final_data.shape

Посмотрим общую информацию для нашего датасета

In [ ]:
final_data.info()

Получился датасет с 30 колонками и 49752 строками. Видно, что в некоторых столбцах присутствуют пропуски. Посмотрим их количество

In [ ]:
final_data.isna().sum()

Видно, что одинаковое количество пропусков в признаках связаных с рассылками. 

In [ ]:
# Заполним все пропуски связанные с рассылками 0
# принимая, что сообщения не направлялись 
final_data.loc[final_data['total_messages'].isna(), [
    'total_messages', 
    'opened_count',
    'clicked_count', 
    'purchased_from_msg',
    'unique_campaigns',
    'open_rate',
    'click_rate',
    'purchase_rate',
    'click_to_open'
     ]] = 0

# Количество дней с последней рассылке зададим 365 дней
final_data.loc[final_data[
    'days_since_last_msg'].isna(), 
    'days_since_last_msg'] = 365

In [ ]:
final_data.isna().sum()

Удалим пропус в столбце `most_frequent_second_last_category`

In [ ]:
final_data.dropna(
    subset=['most_frequent_second_last_category'],
    inplace=True
    )

Мы обработали пропуски

Обработаем выбросы

In [ ]:
def winsorize_series(series, limits=(0.01, 0.01)):
    """Ограничивает выбросы сверху и снизу."""
    lower = series.quantile(limits[0])
    upper = series.quantile(1 - limits[1])
    return series.clip(lower, upper)

cols_to_winsorize = ['total_spent', 'max_purchase', 'avg_interval', 'max_interval']
for col in cols_to_winsorize:
    if col in final_data.columns:
        final_data[col] = winsorize_series(final_data[col])

Выбросы обработаны

### 5.1 Промежуточный вывод

Мы создали новые признаки:
1. days_since_last_purchase - дней с последней покупки

2. purchase_frequency - частота покупок

3. total_spent - всего потрачено

4. avg_purchase_value - средний чек

5. max_purchase - максимальная покупка

6. total_items - всего товаров

7. avg_items_per_order - среднее количество товаров в заказе

8. unique_last_categories - уникальные последние категории

9. unique_second_last_categories - уникальные предпоследние категории

10. most_frequent_last_category - самая частая последняя категория

11. most_frequent_second_last_category - самая частая предпоследняя категория

12. avg_categories_per_purchase - среднее количество категорий на покупку

13. max_categories_in_purchase - максимальное количество категорий в покупке

14. min_categories_in_purchase - минимальное количество категорий в покупке

15. total_categories - всего категорий

16. total_messages - всего сообщений

17. opened_count - количество открытий

18. clicked_count - количество кликов

19. purchased_from_msg - покупок из сообщений

20. unique_campaigns - уникальные кампании

21. days_since_last_msg - дней с последнего сообщения

22. open_rate - процент открытий

23. click_rate - процент кликов

24. purchase_rate - процент покупок

25. click_to_open - конверсия из открытия в клик

26. favorite_weekday - любимый день недели

27. favorite_hour - любимый час

28. weekend_purchases_ratio - доля покупок в выходные


## 6. Подготовка данных для обучения моделей

### 6.1 Корреляционный анализ

Простроим матрицу корреляций

In [ ]:

matrix_data = final_data.copy()
matrix_data.drop('client_id', axis=1, inplace=True)
interval_cols = [
    'days_since_last_purchase',
    'total_spent',
    'avg_purchase_value', 
    'max_purchase', 
    'avg_categories_per_purchase', 
    'total_messages', 
    'unique_campaigns', 
    'days_since_last_msg', 
    'open_rate', 
    'click_rate', 
    'purchase_rate', 
    'click_to_open'
    ]
phik_mat(matrix_data, interval_cols=interval_cols)

По матрице корреляций видно, сильно коррелирующие столбцы: `weekend_purchases_ratio`, `total_categories`, `unique_last_categories`, `unique_second_last_categories`, `most_frequent_categories`, `avg_categories_per_purchase`. А также признаки, которые не имеют корреляцию с целевым признаком: `max_categories_in_purchase`, `purchase_rate`. Удалим данные столбцы для устранения мультиколлинеарности и упрощения расчетов при работе модели

In [ ]:
final_data = final_data.drop([
    'weekend_purchases_ratio',
    'total_categories',
    'unique_last_categories',
    'unique_second_last_categories',
    'most_frequent_last_category',
    'most_frequent_second_last_category',
    'avg_categories_per_purchase',
    'max_categories_in_purchase',
    'purchase_rate',
    'favorite_hour'
], axis=1)

Повторим корреляционный анализ

In [ ]:
matrix_data = final_data.copy()
matrix_data.drop('client_id', axis=1, inplace=True)
interval_cols = [
    'days_since_last_purchase',
    'total_spent',
    'avg_purchase_value', 
    'max_purchase',  
    'total_messages', 
    'unique_campaigns', 
    'days_since_last_msg', 
    'open_rate', 
    'click_rate', 
    'click_to_open'
    ]
phik_mat(matrix_data, interval_cols=interval_cols)

Мы удалили все мультиколлинеарные признаки с показателем более 0.95, а также признаки не влияющие на целевую переменную.

Выведем информацию о полученном датасете

In [ ]:
final_data.info()

### 6.2 Подготовка выборок для обучения

Поменяем индексы на `client_id`

In [ ]:
final_data.set_index('client_id', inplace=True)

Выделим целевой и входные признаки

In [ ]:
X = final_data.drop(['target'], axis=1)
y = final_data['target']

Разделим данные на тренировочную и тестовую выборки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size= TEST_SIZE, 
    random_state=RANDOM_STATE,
    stratify=y
)

In [ ]:
print(
    f'Тренировочная выборка:{X_train.shape},{y_train.shape} \
    \nТестовая выборка:{X_test.shape}, {y_test.shape}'
)

### 6.3 Пайплайн для предобработки данных

Подготовим списки со столбцами для обработки

In [ ]:
ohe_columns = [
    'favorite_weekday'
    ]
num_columns = [
    'days_since_last_purchase', 
    'purchase_frequency',
    'total_spent',
    'avg_purchase_value', 
    'max_purchase',
    'total_items',
    'avg_items_per_order',
    'min_categories_in_purchase',
    'total_messages',
    'opened_count',
    'clicked_count',
    'purchased_from_msg',
    'unique_campaigns',
    'days_since_last_msg',
    'open_rate',
    'click_rate',
    'click_to_open'
    ]

Создадим пайплайн для предобработки данных

In [ ]:
ohe_pipe = Pipeline(
    [
        ('simpleImputer_ohe', SimpleImputer(
            missing_values=np.nan, 
            strategy='most_frequent'
        )),
        ('ohe', OneHotEncoder(
            drop='first', 
            handle_unknown='ignore', 
            sparse_output=False))
    ]
)

Создадим единный пайплайн для предобработки

In [ ]:
data_preprocessor = ColumnTransformer(
    [('ohe', ohe_pipe, ohe_columns),
     ('num', RobustScaler(), num_columns)
    ], 
    remainder='passthrough'
)

Создадим единный пайплайн для предобработки данных и обучения модели

In [ ]:
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

### 6.4 Промежуточный вывод

При корреляционном анализе была обнаружена мультиколлинеарность между входными признаками, а также незначимые признаки, которые в дальнейшем были удалены. Данные разделены на тренировочную и тестовую выборки, подготовлен пайплайн для предобработки и обучения моделей.

## 7. Обучение моделей

Обучим модели:
1) DecisionTreeClassifier
2) KNeighborsClassifier
3) RandomForestClassifier
4) LGBMRegressor
5) XGBClassifier

Обучение будем проводить по каждой модели отдельно, для большего контроля.

In [ ]:
scoring={
    'roc_auc': 'roc_auc',
    'recall': 'recall',
    'precision': 'precision',
    'f1': 'f1'
}

**DesisionTreeClassifier**

Cоздадим словарь гиперпараметров 

In [ ]:
param_grid_1 = {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 20),
        'models__max_features': range(2, 20),
        'models__min_samples_split': range(2, 20),
        'models__class_weight': ['balanced', None]
    }

Обучим модель

In [ ]:
start_time = time.time()

randomized_search_1 = RandomizedSearchCV(
    pipe_final, 
    param_grid_1, 
    cv=5,
    scoring=scoring,
    refit='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1, 
    error_score='raise'
)

randomized_search_1.fit(X_train, y_train)

end_time = time.time()
f"Время подбора гиперпараметров: {(end_time - start_time)/60:.2f} минут"

Найдем метрики на кросс-валидации, сортировка по ROC-AUC

In [ ]:
cv_results_1 = pd.DataFrame(randomized_search_1.cv_results_)
print_best_metrics(cv_results_1)

**KNeighborsClassifier**

In [ ]:
param_grid_2 = {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(2, 30),
        'models__weights': ['uniform', 'distance'],
        'models__metric': ['euclidean', 'manhattan', 'minkowski'],
        'models__leaf_size': range(10, 30) 
    }

In [ ]:
start_time = time.time()

randomized_search_2 = RandomizedSearchCV(
    pipe_final, 
    param_grid_2, 
    cv=5,
    scoring=scoring,
    refit='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1, 
    error_score='raise'
)

randomized_search_2.fit(X_train, y_train)

end_time = time.time()
f"Время подбора гиперпараметров: {(end_time - start_time)/60:.2f} минут"

In [ ]:
cv_results_2 = pd.DataFrame(randomized_search_2.cv_results_)
print_best_metrics(cv_results_2)

**RandomForestClassifier**

In [ ]:
param_grid_3 = {    
    'models': [RandomForestClassifier(random_state=RANDOM_STATE)],
    'models__n_estimators': [100, 200, 300],
    'models__max_depth': [5, 10, 20, None],
    'models__min_samples_split': [2, 5, 10, 20],
    'models__min_samples_leaf': [1, 2, 4, 10],
    'models__class_weight': ['balanced', None],
    'preprocessor__num': ['passthrough'] 
}

In [ ]:
start_time = time.time()

randomized_search_3 = RandomizedSearchCV(
    pipe_final, 
    param_grid_3, 
    cv=5,
    scoring=scoring,
    refit='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1, 
    error_score='raise'
)

randomized_search_3.fit(X_train, y_train)

end_time = time.time()
f"Время подбора гиперпараметров: {(end_time - start_time)/60:.2f} минут"

In [ ]:
cv_results_3 = pd.DataFrame(randomized_search_3.cv_results_)
print_best_metrics(cv_results_3)

**LGBMRegressor**

In [ ]:
param_grid_4 = {
    'models': [LGBMClassifier(random_state=RANDOM_STATE)],
    'models__n_estimators': [125, 250, 370],
    'models__max_depth': range(-1, 16),
    'models__learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4],
    'models__num_leaves': range(15, 120),
    'models__min_child_samples': range(1, 15),
    'models__min_split_gain': [0.0, 0.1],
    'models__class_weight': ['balanced', None],
    'models__verbose': [-1],
    'preprocessor__num': ['passthrough']
}

In [ ]:
start_time = time.time()

randomized_search_4 = RandomizedSearchCV(
    pipe_final, 
    param_grid_4, 
    cv=5,
    scoring=scoring,
    refit='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1, 
    error_score='raise'
)

randomized_search_4.fit(X_train, y_train)

end_time = time.time()
f"Время подбора гиперпараметров: {(end_time - start_time)/60:.2f} минут"

In [ ]:
cv_results_4 = pd.DataFrame(randomized_search_4.cv_results_)
print_best_metrics(cv_results_4)

**XGBClassifier**

In [ ]:
param_grid_5 = {    
        'models': [XGBClassifier(random_state=RANDOM_STATE)],
        'models__iterations': [300, 500, 800, 1200],
        'models__depth': range(3, 10),
        'models__learning_rate': [0.03, 0.05, 0.1],
        'models__l2_leaf_reg': range(1, 10),
        'models__scale_pos_weight': [20, 40, 51, 70, 100, 150],
        'models__border_count': range(64, 254),
        'models__random_strength': [0, 1, 2],
        'models__bagging_temperature': [0, 0.5, 1],
        'preprocessor__num': ['passthrough']
    }

In [ ]:
start_time = time.time()

randomized_search_5 = RandomizedSearchCV(
    pipe_final, 
    param_grid_5, 
    cv=5,
    scoring=scoring,
    refit='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1, 
    error_score='raise'
)

randomized_search_5.fit(X_train, y_train)

end_time = time.time()
f"Время подбора гиперпараметров: {(end_time - start_time)/60:.2f} минут"

In [ ]:
cv_results_5 = pd.DataFrame(randomized_search_5.cv_results_)
print_best_metrics(cv_results_5)

### 7.1 Промежуточный вывод

Было обучено 5 моделей классификации: 
1) DecisionTreeClassifier
2) KNeighborsClassifier
3) RandomForestClassifier
4) LGBMRegressor
5) XGBClassifier

Лучший результат на кросс-валидации по метрике ROC-AUC показала модель RandomForestClassifier, вторая по метрике модель LGBMRegressor.

## 8. Улучшение модели и максимизация метрики ROC-AUC

Мы нашли две лучшие модели, улучшим модель перебором большего количества вариантов гиперпараметров для улучшения метрики ROC-AUC

Улучшим модель RandomForestClassifier

In [ ]:
param_grid_best_1 = {
    'model': [RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1)],  
    'model__n_estimators': [100, 200, 300, 400],  
    'model__max_depth': [3, 5, 7, 10, 15, 20, None],  
    'model__min_samples_split': [2, 5, 10, 20, 30], 
    'model__min_samples_leaf': [1, 2, 4, 8, 12, 20],  
    'model__max_features': ['sqrt', 'log2', 0.5, 0.7, None], 
    'model__max_samples': [0.6, 0.7, 0.8, 0.9, None],    
    'model__class_weight': [
        'balanced',
        'balanced_subsample',
        {0: 1, 1: 10},  
        {0: 1, 1: 20},
        None
    ],    
    'model__criterion': ['gini', 'entropy'],
    'model__bootstrap': [True, False],  
    'preprocessor__num': ['passthrough']
}

In [ ]:
start_time = time.time()

randomized_search_best_1 = RandomizedSearchCV(
    pipe_final, 
    param_grid_best_1, 
    cv=5,
    scoring=scoring,
    refit='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1, 
    error_score='raise',
    n_iter=50
)

randomized_search_best_1.fit(X_train, y_train)

end_time = time.time()
f"Время подбора гиперпараметров: {(end_time - start_time)/60:.2f} минут"

In [ ]:
cv_results_best_1 = pd.DataFrame(randomized_search_best_1.cv_results_)
print_best_metrics(cv_results_best_1)

При большем переборе гиперпараметров модель не улучшила метрику ROC-AUC

Проверим модель LGBMСlassifier

In [ ]:
param_grid_best_2 = {
    'models': [LGBMClassifier(random_state=RANDOM_STATE, n_jobs=-1, verbose=-1)],
    'models__n_estimators': [100, 200, 300, 500],
    'models__max_depth': [3, 5, 7, 10, -1],
    'models__learning_rate': [0.01, 0.05, 0.1, 0.15],
    'models__num_leaves': [15, 31, 63, 127],
    'models__min_child_samples': [10, 20, 30, 50],
    'models__min_split_gain': [0.0, 0.001, 0.01],
    'models__scale_pos_weight': [40, 49, 55, 60, 75],
    'models__reg_alpha': [0.0, 0.01, 0.1],
    'models__reg_lambda': [0.0, 0.1, 1.0],
    'models__subsample': [0.7, 0.8, 0.9],
    'models__colsample_bytree': [0.7, 0.8, 0.9],
    'preprocessor__num': ['passthrough']
}

In [ ]:
start_time = time.time()

randomized_search_best_2 = RandomizedSearchCV(
    pipe_final, 
    param_grid_best_2, 
    cv=5,
    scoring=scoring,
    refit='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1, 
    error_score='raise',
    n_iter=50
)

randomized_search_best_2.fit(X_train, y_train)

end_time = time.time()
f"Время подбора гиперпараметров: {(end_time - start_time)/60:.2f} минут"

In [ ]:
cv_results_best_2 = pd.DataFrame(randomized_search_best_2.cv_results_)
print_best_metrics(cv_results_best_2)

### 8.1 Промежуточный вывод

Мы выделили 2 лучшие модели и улучшили модель LGBMСlassifier, подняв метрику ROC-AUC с 0,7033 до 0,7280. Улучшение модели RandomForestClassifier не дало результата, метрика не изменилась и равняется 0.7215

## 9. Тестирование лучшей модели

Сохраним лучшую модель 

In [ ]:
best_model = randomized_search_best_2.best_estimator_

Выполним предсказание на тестовых данных

In [ ]:
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1] 

Найдем метрику ROC-AUC на тестовых данных и визуализируем кривую

In [ ]:
roc_auc = roc_auc_score(y_test, y_pred_proba)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, 
         label=f'ROC кривая (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', 
         label='Случайная модель (AUC = 0.5)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nПлощадь под ROC-кривой (AUC): {auc(fpr, tpr):.4f}")

ROC-AUC на тестовых данных равен 0.7467, что является хорошим показателям (требование ROC-AUC более 0.7)

Выведем параметры лучшей модели

In [ ]:
print(best_model)

Построим матрицу ошибок

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print("Матрица ошибок:")
print(cm)

# Визуализация матрицы ошибок
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Матрица ошибок')
plt.ylabel('Истинные значения')
plt.xlabel('Предсказанные значения')
plt.show()

По матрице ошибок видно, что у нас малое количество ложноотрицетельных примеров, но большое количество ложноположительных примеров. Это можно объяснить дисбалансом между классами. Бизнес ожидает больший отклик от клиентов, но на самом деле большая часть из них не совершит покупку.

Построим важность признаков

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Минимальный рабочий вариант без permutation importance
lgbm_model = best_model.named_steps['models']
preprocessor = best_model.named_steps['preprocessor']

# Получаем имена признаков после препроцессинга
feature_names = preprocessor.get_feature_names_out()

# Получаем важность признаков (только split)
feature_importances = lgbm_model.feature_importances_

# Создаем DataFrame
feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importances
}).sort_values('importance', ascending=False)

# Выводим результаты
print("Топ-30 важных признаков:")
print(feature_importance_df.head(30))

# Простая визуализация
plt.figure(figsize=(12, 10))
top_features = feature_importance_df.head(20).sort_values('importance', ascending=True)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Feature Importance (split)')
plt.title('Top 20 Feature Importances')
plt.tight_layout()
plt.savefig('simple_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

По важности признаков мы определили 10 самых важных

1. days_since_last_purchase - дней с последней покупки

2. total_messages - всего сообщений

3. avg_purchase_value - средний чек

4. unique_campaigns - уникальные кампании

5. purchase_frequency - частота покупок

6. total_items - всего товаров

7. days_since_last_msg - дней с последнего сообщения

8. max_purchase - максимальная покупка

9. total_spent - всего потрачено

10. purchased_from_msg - покупок из сообщений


## 10. Вывод

- Данные заружены, установлена настройка при чтении файла, для обработки столбцов с датами. Наименования столбцов имеют "змеиный" регистр. Все данные имеют корректный тип данных.

- В данных отсутствуют пропущенные значения. В датасете `purchases` удалены аномальные значения цены единицы товара (установлена верхняя граница 10000), а также удалены полные дубликаты - 72900 строк из 202208.

- Мы создали новые признаки:

        1. days_since_last_purchase - дней с последней покупки

        2. purchase_frequency - частота покупок

        3. total_spent - всего потрачено

        4. avg_purchase_value - средний чек

        5. max_purchase - максимальная покупка

        6. total_items - всего товаров

        7. avg_items_per_order - среднее количество товаров в заказе

        8. unique_last_categories - уникальные последние категории

        9. unique_second_last_categories - уникальные предпоследние категории

        10. most_frequent_last_category - самая частая последняя категория

        11. most_frequent_second_last_category - самая частая предпоследняя категория

        12. avg_categories_per_purchase - среднее количество категорий на покупку

        13. max_categories_in_purchase - максимальное количество категорий в покупке

        14. min_categories_in_purchase - минимальное количество категорий в покупке

        15. total_categories - всего категорий

        16. total_messages - всего сообщений

        17. opened_count - количество открытий

        18. clicked_count - количество кликов

        19. purchased_from_msg - покупок из сообщений

        20. unique_campaigns - уникальные кампании

        21. days_since_last_msg - дней с последнего сообщения

        22. open_rate - процент открытий

        23. click_rate - процент кликов

        24. purchase_rate - процент покупок

        25. click_to_open - конверсия из открытия в клик

        26. favorite_weekday - любимый день недели

        27. favorite_hour - любимый час

        28. weekend_purchases_ratio - доля покупок в выходные

- При корреляционном анализе была обнаружена мультиколлинеарность между входными признаками, а также незначимые признаки, которые в дальнейшем были удалены. Данные разделены на тренировочную и тестовую выборки, подготовлен пайплайн для предобработки и обучения моделей.

- Было обучено 5 моделей классификации: 
    1) DecisionTreeClassifier
    2) KNeighborsClassifier
    3) RandomForestClassifier
    4) LGBMRegressor
    5) XGBClassifier

    Лучший результат на кросс-валидации по метрике ROC-AUC показала модель RandomForestClassifier, вторая по метрике модель LGBMRegressor.

- Мы выделили 2 лучшие модели и улучшили модель LGBMСlassifier, подняв метрику ROC-AUC с 0,7033 до 0,7280. Улучшение модели RandomForestClassifier не дало результата, метрика не изменилась и равняется 0.7215

- ROC-AUC на тестовых данных равен 0.7467, что является хорошим показателям (требование ROC-AUC более 0.7). У нас малое количество ложноотрицетельных примеров, но большое количество ложноположительных примеров. Это можно объяснить дисбалансом между классами. Бизнес ожидает больший отклик от клиентов, но на самом деле большая часть из них не совершит покупку.

- По важности признаков мы определили 10 самых важных

        1. days_since_last_purchase - дней с последней покупки

        2. total_messages - всего сообщений

        3. avg_purchase_value - средний чек

        4. unique_campaigns - уникальные кампании

        5. purchase_frequency - частота покупок

        6. total_items - всего товаров

        7. days_since_last_msg - дней с последнего сообщения

        8. max_purchase - максимальная покупка

        9. total_spent - всего потрачено

        10. purchased_from_msg - покупок из сообщений
